In [ ]:
# install packages
!pip install --quiet pycosat
!pip install --quiet python-sat[pblib,aiger]
!pip install --quiet mip
!pip install --quiet shapely

## Using SAT-solvers

In [ ]:
from pycosat import solve

print(solve([[1, -2, -3], [3, -1], [2, 3]]))
print(solve([[-2, 3], [2, 1], [-3], [3, -1]]))

### Solving the N Queens Puzzle

In [ ]:
from itertools import combinations, product
from pycosat import solve

n = 10
clauses = []


# converts a pair of integers into a unique positive integer
def varnum(row, column):
    assert row in range(n) and column in range(n)
    return row * n + column + 1


# each row contains at least one queen
for row in range(n):
    clauses.append([varnum(row, column) for column in range(n)])

# each row contains at most one queen
for row in range(n):
    for column1, column2 in combinations(range(n), 2):
        clauses.append([-varnum(row, column1), -varnum(row, column2)])

# each column contains at most one queen
for column in range(n):
    for row1, row2 in combinations(range(n), 2):
        clauses.append([-varnum(row1, column), -varnum(row2, column)])

# no two queens stay on the same diagonal
for r1, c1, r2, c2 in product(range(n), repeat=4):
    if r1 == r2:
        continue

    if abs(r1 - r2) == abs(c1 - c2):
        clauses.append([-varnum(r1, c1), -varnum(r2, c2)])


model = solve(clauses)

for row in range(n):
    for column in range(n):
        print('X' if varnum(row, column) in model else '*', end='')
    print()

### Solving the Sudoku Puzzle

In [ ]:
from itertools import combinations, product
import pycosat


def varnum(row, column, digit):
    assert row in range(1, 10) and column in range(1, 10)
    assert digit in range(1, 10)
    return 100 * row + 10 * column + digit


def exactly_one_of(literals):
    clauses = [[l for l in literals]]
    for pair in combinations(literals, 2):
        clauses.append([-l for l in pair])
    return clauses


def one_digit_in_every_cell():
    clauses = []
    for (row, column) in product(range(1, 10), repeat=2):
        clauses += \
            exactly_one_of([varnum(row, column, digit)
                            for digit in range(1, 10)])
    return clauses


def one_digit_in_every_row():
    clauses = []
    for (row, digit) in product(range(1, 10), repeat=2):
        clauses += \
            exactly_one_of([varnum(row, column, digit)
                            for column in range(1, 10)])
    return clauses


def one_digit_in_every_column():
    clauses = []
    for (column, digit) in product(range(1, 10), repeat=2):
        clauses += \
            exactly_one_of([varnum(row, column, digit)
                            for row in range(1, 10)])
    return clauses


def one_digit_in_every_block():
    clauses = []
    for (row, column) in product([1, 4, 7], repeat=2):
        for digit in range(1, 10):
            clauses += exactly_one_of(
                [varnum(row + a, column + b, digit)
                 for (a, b) in product(range(3), repeat=2)]
            )
    return clauses


def solve_puzzle(puzzle):
    assert len(puzzle) == 9
    assert all(len(row) == 9 for row in puzzle)

    clauses = []
    clauses += one_digit_in_every_cell()
    clauses += one_digit_in_every_row()
    clauses += one_digit_in_every_column()
    clauses += one_digit_in_every_block()

    for (row, column) in product(range(1, 10), repeat=2):
        if puzzle[row - 1][column - 1] != "*":
            digit = int(puzzle[row - 1][column - 1])
            assert digit in range(1, 10)
            clauses += [[varnum(row, column, digit)]]

    solution = pycosat.solve(clauses)
    if isinstance(solution, str):
        print("No reply")
        exit()

    assert isinstance(solution, list)

    for row in range(1, 10):
        for column in range(1, 10):
            for digit in range(1, 10):
                if varnum(row, column, digit) in solution:
                    print(digit, end="")
        print()


difficult_puzzle = [
    "8********",
    "**36*****",
    "*7**9*2**",
    "*5***7***",
    "****457**",
    "***1***3*",
    "**1****68",
    "**85***1*",
    "*9****4**"
]

solve_puzzle(difficult_puzzle)

### Solving the N Queens Puzzle Using Pysat Module

In [ ]:
from itertools import product
from pysat.solvers import Solver
from pysat.card import *
from pysat.formula import IDPool, CNF

n = 20
pool, clauses = IDPool(), CNF()

for row in range(n):
    clauses.extend(CardEnc.equals(lits=[pool.id((row, column)) for column in range(n)], bound=1, vpool=pool))

for column in range(n):
    clauses.extend(CardEnc.equals(lits=[pool.id((row, column)) for row in range(n)], bound=1, vpool=pool))

for row, column in product(range(n), repeat=2):
    clauses.extend(CardEnc.atmost(
        lits=[pool.id((r, c)) for r, c in product(range(n), repeat=2) if r - c == row - column], bound=1, vpool=pool))
    clauses.extend(CardEnc.atmost(
        lits=[pool.id((r, c)) for r, c in product(range(n), repeat=2) if r + c == row + column], bound=1, vpool=pool))

solver = Solver(bootstrap_with=clauses)
solver.solve()
model = solver.get_model()

for row in range(n):
    for column in range(n):
        print('X' if pool.id((row, column)) in model else '*', end='')
    print()

### Solving the 16 Diagonals Puzzle

In [ ]:
from itertools import combinations, product
from pysat.solvers import Solver
from pysat.card import *
from pysat.formula import IDPool, CNF
from shapely.geometry import LineString


n, num_diagonals = 5, 16

segments = {}
pool, formula = IDPool(), CNF()

for x, y in product(range(n + 1), repeat=2):
    for a, b in product((-1, 1), repeat=2):
        if x + a in range(n + 1) and y + b in range(n + 1):
            segments[x, y, x + a, y + b] = pool.id((x, y, x + a, y + b))

for s1, s2 in combinations(segments, 2):
    if LineString([s1[:2], s1[2:]]).intersects(LineString([s2[:2], s2[2:]])):
        formula.append([-segments[s1], -segments[s2]])

formula.extend(CardEnc.atleast(lits=segments.values(), bound=num_diagonals, vpool=pool))

solver = Solver(bootstrap_with=formula)
solver.solve()
model = solver.get_model()

for s in segments:
    if segments[s] in model:
        print(s)

## Using ILP Solvers

In [ ]:
from mip import *

model = Model(solver_name='cbc')
model.verbose = False

m = model.add_var(var_type=INTEGER)
d = model.add_var(var_type=INTEGER)

model += (m <= 500)
model += (d <= 200)
model += (m + d <= 600)

model.objective = maximize(10 * m + 30 * d)
model.optimize()

print(model.objective_value)

### Solving the 16 Diagonals Puzzle

In [ ]:
from itertools import combinations, product
from mip.model import *
from shapely.geometry import LineString

n = 5

model = Model(solver_name='cbc')
model.verbose = False

segments = {}
for i, j in product(range(n + 1), repeat=2):
    for delta_i, delta_j in product((-1, 1), repeat=2):
        if i + delta_i in range(n + 1) and \
                j + delta_j in range(n + 1):
            segments[i, j, i + delta_i, j + delta_j] = \
                model.add_var(var_type=mip.BINARY)

for s1, s2 in combinations(segments, 2):
    if LineString([s1[:2], s1[2:]]).intersects(
            LineString([s2[:2], s2[2:]])
    ):
        model += segments[s1] + segments[s2] <= 1

model.objective = maximize(xsum(segments.values()))
model.optimize()

print(model.objective_value)
for s in segments:
    if abs(segments[s].x) > 1e-6:
        print(s)